In [5]:
!pip install ktrain

     |████████████████████████████████| 25.2MB 1.4MB/s 
     |████████████████████████████████| 983kB 51.8MB/s 
     |████████████████████████████████| 245kB 50.0MB/s 
     |████████████████████████████████| 573kB 44.9MB/s 
     |████████████████████████████████| 1.0MB 50.1MB/s 
     |████████████████████████████████| 870kB 51.7MB/s 
     |████████████████████████████████| 3.7MB 49.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.12.3-cp36-none-any.whl size=25228500 sha256=3aa295f8fa75a5edc93c13aa1edf5b79a2905093dacd0efb7295e40def688268
  Stored in directory: /root/.cache/pip/wheels/93/8f/77/a036a53942ede8e46e333d39b6abd9aaf3ab91aaf96d83e70f
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=e33c164a812a7a96332f228aaf08874a777e84387410fc96e26130c3371b2df7
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl siz

In [0]:
import os
os.environ['DISABLE_V2_BEHAVIOR'] = '1'

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [6]:
import ktrain
from ktrain import text

Instructions for updating:
non-resource variables are not supported in the long term
Using DISABLE_V2_BEHAVIOR with TensorFlow


##Step 1 : Define a dataset
There are 2 types of dataset which can be fed to bert sequence tagging model


1.   entities_from_txt
2.   entities_from_array

We are choosing the 2nd since we need to create a custom dataset



In [0]:
x_train = [
           ['30kg', 'apples', '90rs', 'each', 'kg'],
           ['70', 'kg', 'pears', '60', 'rs', 'each', 'kg'],
           ['I', 'want', 'to', 'sell', '50kg', 'of', 'wheat', 'for', '400', 'rupees'],
           ['sell', '60', 'kg', 'of', 'rice', 'for', '300rupees'],
           ['30kg', 'wheat', '90rs', 'each'],
           ['I', 'want', 'to', 'sell', '50kg', 'bajra', 'for', '400', 'rupees'],
           ['20kg', 'dragonfruit', '10rs', 'per', 'kg'],
           ['I', 'have', 'fresh', 'carrots', 'for', '50rupees', '20kg']
]

In [0]:
y_train = [
          ['weight', 'product', 'price', '0', 'weight'],
          ['amt', 'weight', 'product', 'amt', 'price', '0', 'weight'],
          ['0', '0', '0', '0' ,'weight', '0', 'product', '0', 'amt', 'price'],
          ['0', 'amt', 'weight', '0', 'product', '0', 'price'],
          ['weight', 'product', 'price', '0'],
          ['0', '0', '0', '0', 'weight', 'product', '0', 'amt', 'price'],
          ['weight', 'product', 'price', '0', 'weight'],
          ['0', '0', '0', 'product', '0', 'price', 'weight']     
]

In [12]:
(trn, val, preproc) = text.entities_from_array(x_train, y_train)

training data sample:
   SentenceID    Word      Tag
0           0    30kg   weight
1           0  apples  product
2           0    90rs    price
3           0    each        0
4           0      kg   weight
Number of sentences:  8
Number of words in the dataset:  30
Tags: ['price', '0', 'weight', 'product', 'amt']
Number of Labels:  5
Longest sentence: 10 words


##STEP 2: Define a Model
The print_sequence_taggers function shows that, as of this writing, ktrain currently supports both Bidirectional LSTM-CRM and Bidirectional LSTM as base models for sequence tagging. Theses base models can be used with different embedding schemes.

In [13]:
text.print_sequence_taggers()

bilstm: Bidirectional LSTM (https://arxiv.org/abs/1603.01360)
bilstm-bert: Bidirectional LSTM w/ BERT embeddings
bilstm-crf: Bidirectional LSTM-CRF  (https://arxiv.org/abs/1603.01360)
bilstm-elmo: Bidirectional LSTM w/ Elmo embeddings [English only]
bilstm-crf-elmo: Bidirectional LSTM-CRF w/ Elmo embeddings [English only]


In [14]:
WV_URL = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz'
model = text.sequence_tagger('bilstm-crf', preproc, wv_path_or_url=WV_URL)

embedding schemes employed:
	word embeddings initialized with fasttext word vectors (cc.en.300.vec.gz)

pretrained word embeddings will be loaded from:
	https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
downloading pretrained word vectors to /root/ktrain_data ...
[██████████████████████████████████████████████████]
extracting pretrained word vectors...
done.

cleanup downloaded zip...
done.

loading pretrained word vectors...this may take a few moments...


In [0]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=128)


##STEP 3: Train and Evaluate the Model
Here, we will train for a single epoch using an initial learning rate of 0.01 with gradual decay using cosine annealing (via the cycle_len=1) parameter and see how well we do. The learning rate of 0.01 is determined with the learning-rate-finder (i.e., lr_find).

In [25]:
learner.fit(1e-2, 1, cycle_len=1)

1/1 [==============================] - 0s 105ms/step - loss: 0.9977 - val_loss: 0.0413


In [26]:
learner.validate()

   F1: 100.00
           precision    recall  f1-score   support

    price       1.00      1.00      1.00         1
   weight       1.00      1.00      1.00         2
        0       1.00      1.00      1.00         1
  product       1.00      1.00      1.00         1

micro avg       1.00      1.00      1.00         5
macro avg       1.00      1.00      1.00         5



1.0

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [31]:
predictor.predict('I have 60 kg dragonfruit for 20rs each')

[('I', '0'),
 ('have', '0'),
 ('60', 'amt'),
 ('kg', 'weight'),
 ('dragonfruit', 'product'),
 ('for', '0'),
 ('20rs', 'price'),
 ('each', '0')]

In [32]:
predictor.predict('20 kg apples rs 30 each')

[('20', '0'),
 ('kg', 'weight'),
 ('apples', 'product'),
 ('rs', 'price'),
 ('30', 'price'),
 ('each', '0')]